In [ ]:
import cv2
import numpy as np
import pandas as pd

In [ ]:

def paint_rectangle(image_path, top_left, bottom_right, output_path="output_image.png"):
    """
    Paints a rectangle in red over the image.

    Parameters:
    - image_path (str): Path to input image.
    - top_left (tuple): (x, y) of top-left corner.
    - bottom_right (tuple): (x, y) of bottom-right corner.
    - output_path (str): Path to save modified image.
    """
    img = cv2.imread(image_path)

    if img is None:
        raise IOError("Error: Could not read image.")

    # BGR format → Red = (0,0,255)
    cv2.rectangle(img, top_left, bottom_right, (0, 0, 255), thickness=-1)  # -1 = filled rectangle

    cv2.imwrite(output_path, img)

    cv2.imshow("Rectangle painted", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
paint_rectangle('WIN_20250907_21_47_03_Pro.jpg', (600, 100), (700, 150))

In [ ]:

def extract_rgb_signals_rect(video_path, rect, output_csv="rgb_signals.csv"):
    """
    Extracts R, G, B signals from all pixels inside a rectangle across video frames.

    Parameters:
    - video_path (str): Path to input video.
    - rect (tuple): (x1, y1, x2, y2) defining the rectangle (top-left to bottom-right)
    - output_csv (str): Output CSV filename.
    """
    x1, y1, x2, y2 = rect
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise IOError("Error opening video file.")

    signals = {}
    idx_map = {}  # map for pixel index
    idx = 0
    for y in range(y1, y2):
        for x in range(x1, x2):
            signals[f"Pixel{idx}_R"] = []
            signals[f"Pixel{idx}_G"] = []
            signals[f"Pixel{idx}_B"] = []
            idx_map[idx] = (x, y)
            idx += 1

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        for idx, (x, y) in idx_map.items():
            b, g, r = frame[y, x]
            signals[f"Pixel{idx}_R"].append(r)
            signals[f"Pixel{idx}_G"].append(g)
            signals[f"Pixel{idx}_B"].append(b)

        frame_idx += 1

    cap.release()

    df = pd.DataFrame(signals)
    df.to_csv(output_csv, index=False)

    return df